<a href="https://colab.research.google.com/github/JayNguyen-123/Brain_Tumor_Detection_with_Pytorch/blob/main/Pytorch_Brain_tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check if CUDA is available
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")

### Convolution Neural Network for Brain Tumor Detection and Diagnosis


In [ ]:
!pip install split-folders
!pip install torch-summary


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style='darkgrid')
import copy
import os
import torch
from PIL import Image
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from torchvision import utils
from torchvision.datasets import ImageFolder
import splitfolders
from torchsummary import summary
import torch.nn.functional as F
import pathlib
from sklearn.metrics import confusion_matrix, classification_report
import itertools
from tqdm.notebook import trange, tqdm
from torch import optim
import warnings
warnings.filterwarnings('ignore')


### Introduction
Deep Learning has emerged as a powerful tool in the field of medical imaging and has shown great potential in aiding the health community in the detection and diagnosis of brain tumors. By leveraging deep learning algorithms, we can analyze medical images, such as MRI or CT scans, with unprecedented accuracy and efficiency.
By leveraging the power of neural networks, we can enhance the accuracy, efficiency, and understanding of brain tumor imaging, ultimately leading to improved patient care and outcomes in the field of neuro-oncology.


### Dataset


In [ ]:
labels_df = pd.read_csv('/content/metadata.csv')
print(labels_df.head().to_markdown())


|    |   Unnamed: 0 | image           | class   | format   | mode   | shape         |
|---:|-------------:|:----------------|:--------|:---------|:-------|:--------------|
|  0 |            0 | Cancer (1).jpg  | tumor   | JPEG     | RGB    | (512, 512, 3) |
|  1 |            1 | Cancer (1).png  | tumor   | PNG      | L      | (300, 240)    |
|  2 |            2 | Cancer (1).tif  | tumor   | TIFF     | RGB    | (256, 256, 3) |
|  3 |            3 | Cancer (10).jpg | tumor   | JPEG     | RGB    | (512, 512, 3) |
|  4 |            4 | Cancer (10).tif | tumor   | TIFF     | RGB    | (256, 256, 3) |


In [ ]:
os.listdir('')


In [ ]:
labels_df.shape


### Data Preparation
We need to evaluate the model on validation datasets to track the model's performance during training. Then, Let's use 20% of the dataset for the Validation set and use the rest as the Training set, so we have an 80/20 split!

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# dataset path
data_dir = ('')
data_dir = pathlib.Path(data_dir)

# # splitting dataset to train_set, val_set and test_set
splitfolders.ratio(data_dir, output='brain', seed=20, ratio=(0.8, 0.2))

# new dataset path
data_dir = ''
data_dir = pathlib.Path(data_dir)


### Image Augmentation Definitions

In [ ]:
# define transformation
transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# define an object of the custom dataset for the train and validation
train_set = torchvision.datasets.ImageFolder(data_dir.joinpath('train'), transform=transform)
train_set.transform
val_set = torchvision.datasets.ImageFolder(data_dir.joinpath('Val'), transform=transform)
val_set.transform


In [ ]:
import matplotlib.pyplot as plt

# visualization some images from the train set
CLA_label = {
    0: "Brain Tumor",
    1: "Healthy"
}

figure = plt.figure(figsize=(10,10))
cols, rows = 4,4
for i in range(1, cols*rows + 1):
  sample_idx = torch.randint(len(train_set), size=(1,)).item()
  img, label = train_set[sample_idx]
  figure.add_subplot(rows, cols, i)
  plt.title(CLA_label[label])
  plt.axis("off")
  img_np = img.numpy().transpose((1, 2, 0))

  # clip pixel values to [0,1]
  img_valid_range = np.clip(img_np, 0, 1)
  plt.imshow(img_valid_range)
  plt.suptitle("Brain Images", y=0.95)

plt.show()



### Creating Dataloader


In [ ]:
# import and load train, validation
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=2)



In [ ]:
# print shape for training data and validation data
for key, value in {'Traning data': train_loader, 'Validation data': val_loader}.items():
  for X, y in value:
    print(f"{key}:")
    print(f"Shape of X : {X.shape}")
    print(f"Shape of y : {y.shape} {y.dtype}\n")
    break



### Define Brain Tumor Classifier


In [ ]:
def find_conv2d_shape(hin, win, conv, pool=2):
  # get conv arguments
  kernel_size = conv.kernel_size
  stride = conv.stride
  padding = conv.padding
  dilation = conv.dilation

  hout = np.floor((hin+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
  wout = np.floor((win+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)

  if pool:
    hout/=pool
    wout/=pool

  return int(hout), int(wout)



In [ ]:
# define Architecture for CNN model
class CNN_Tumor(nn.Module):
  # network initialization
  def __init__(self, params):
    super(CNN_Tumor, self).__init__()

    Cin, Hin, Win = params["shape in"]
    init_f = params["initial_filter"]
    num_fc1 = params["num_fc1"]
    num_classes = params["num_classes"]
    self.dropout_rate = params["dropout_rate"]

    # convolution layers
    self.conv1 = nn.Conv2d(Cin, init_f, kernel_size=3)
    h,w = find_conv2d_shape(Hin,Win,self.conv1)

    self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3)
    h,w = find_conv2d_shape(h,w,self.conv2)

    self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3)
    h,w = find_conv2d_shape(h,w,self.conv3)

    self.conv4 = nn.Conv2d(4*init_f, 8*init_f, kernel_size=3)
    h,w = find_conv2d_shape(h,w,self.conv4)

    # compute the flatten size
    self.num_flatten = h*w*8*init_f
    self.fc1 = nn.Linear(self.num_flatten, num_fc1)
    self.fc2 = nn.Linear(num_fc1, num_classes)

def forward(self, X):

  # convolution & pool layers
  X = F.relu(self.conv1(X));
  X = F.max_pool2d(X, 2 ,2)
  X = F.relu(self.conv2(X))
  X = F.max_pool2d(X, 2, 2)
  X = F.relu(self.conv3(X))
  X = F.max_pool2d(X, 2, 2)
  X = F.relu(self.conv4(X))
  X = F.max_pool2d(X, 2, 2)
  X = X.view(-1, self.num_flatten)
  X = F.relu(self.fc1(X))
  X = F.dropout(X, self.dropout_rate)
  X = self.fc2(X)
  return F.log_softmax(X, dim=1)

params_model = {
    "shape_in": (3,256,256),
    "initial_filters": 8,
    "num_fc1": 100,
    "dropout_rate": 0.25,
    "num_classes": 2
}

# create instantiation of network class
cnn_model = CNN_Tumor(params_model)

# define computaion hardware approach(GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = cnn_model.to(device)



In [ ]:
# Model Summary for CNN model
summary(cnn_model, input_size=(3, 256, 256), device=device.type)


### Loss Function Definition

In [ ]:
loss_func = nn.NLLLoss(reduction="sum")

### Optimizer Definition


In [ ]:
opt = optim.Adam(cnn_model.parameters(), lr=3e-4)
lr_scheduler = ReduceLROnPlateau(opt, mode='min',factor=0.5, patience=20,verbose=1)


### Training Model


In [ ]:
# function to get the learning rate
def get_lr(opt):
  for param_group in opt.param_groups:
    return param_group['lr']

# function to compute the loss value per batch of data
def loss_batch(loss_func, output, target, opt=None):
  loss = loss_func(output, target) # to get Loss
  pred = output.argmax(dim=1, keepdim=True) # get Output class
  metric_b = pred.eq(target.view_as(pred)).sum().item() # get performance metrics

  if opt is not None:
    opt.zero_grad()
    loss.backward()
    opt.step()

  return loss.item(), metric_b

# compute the loss value & performance metric for the entire dataset(epoch)
def loss_epoch(model, loss_func, dataset_dl, opt=None):

  run_loss = 0.0
  t_metric = 0.0
  len_data = len(dataset_dl.dataset)

  # internal loop over dataset
  for xb, yb in dataset_dl:
    xb= xb.to(device)
    yb = yb.to(device)
    output = model(xb) # get model output
    loss_b, metric_b = loss_batch(loss_func, output, yb, opt) # get loss per batch
    run_loss += loss_b # update running loss

    if metric_b is not None:
       # update running metric
      t_metric += metric_b

  loss = run_loss/float(len_data) # average loss value
  metric = t_metric/float(len_data) # average metric value

  return loss, metric




### Training Function


In [ ]:
def Train_val(model, params, verbose=False):

  # get the parameters
  epochs = params['epochs']
  loss_func = params['f_loss']
  opt = params['optimizer']
  train_dl = params['train']
  val_dl = params['val']
  lr_scheduler = params['lr_change']
  weight_path = params['weight_path']

  # history of loss values in each epoch
  loss_history = {'train': [], 'val': []}
  # history of metric values in each epoch
  metric_history = {'train': [], 'val': []}
  # a deep copy of weight for the best performing model
  best_model_wts = copy.deepcopy(model.state_dict())
  # initialize best loss to a large value
  best_loss = float('inf')

  # Train model n_epochs
  for epoch in tqdm(range(epochs)):

    # get the learning rate
    current_lr = get_lr(opt)
    if(verbose):
      print('Epoch {}/{}, current lr=[]', format(epoch, epochs-1, current_lr))

    model.train()
    train_loss, train_metric = loss_epoch(model, loss_func, train_dl, opt)

    # collect losses
    loss_history['train'].append(train_loss)
    metric_history['train'].append(train_metric)

    # evaluate model process
    model.eval()
    with torch.no_grad():
      val_loss, val_metric = loss_epoch(model, loss_func, val_dl)

    # store best model
    if(val_loss < best_loss):
      best_loss = val_loss
      best_model_wts = copy.deepcopy(model.state_dict())

      # store weights into a local file
      torch.save(model.state_dict(), weight_path)
      if(verbose):
        print("Copied best model weights")

    # collect loss and metric for a validation dataset
    loss_history['val'].append(val_loss)
    metric_history['val'].append(val_metric)

    # learning rate schedule
    lr_scheduler.step(val_loss)
    if current_lr != get_lr(opt):
      if(verbose):
        print("Loading best model weights!")
      model.load_state_dict(best_model_wts)

    if(verbose):
      print(f"train loss: {train_loss:.6f}, dev loss: {val_loss:.6f}, accuracy: {100*val_metric:.2f}")
      print("-"*10)

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model, loss_history, metric_history



### Training Process


In [ ]:
# Define various parameters used for training and evaluation of a cnn_model
params_train = {
    'train': train_loader,
    'val': val_loader,
    'epochs': 60,
    'optimizer': optim.Adam(cnn_model.parameters(), lr=3e-4),
    'lr_change': ReduceLROnPlateau(opt,
                                   mode='min',
                                   factor=0.5,
                                   patience=20,
                                   verbose=0),
    'f_loss': nn.NLLLoss(reduction='sum'),
    'weight_path': 'weight.pt',
}

# train and validate the model
cnn_model, loss_hist, metric_hist = Train_val(cnn_model, params_train)


### Evaluation Metric Visualization


In [ ]:
# Loss and Accuracy plot
# Convergence History Plot
epochs=params_train["epochs"]
fig,ax = plt.subplots(1,2,figsize=(12,5))

sns.lineplot(x=[*range(1,epochs+1)],y=loss_hist["train"],ax=ax[0],label='loss_hist["train"]')
sns.lineplot(x=[*range(1,epochs+1)],y=loss_hist["val"],ax=ax[0],label='loss_hist["val"]')
sns.lineplot(x=[*range(1,epochs+1)],y=metric_hist["train"],ax=ax[1],label='Acc_hist["train"]')
sns.lineplot(x=[*range(1,epochs+1)],y=metric_hist["val"],ax=ax[1],label='Acc_hist["val"]')


### Confusion Matrix


In [ ]:
# define function For Classification Report
def Ture_and_Pred(val_loader, model):
    i = 0
    y_true = []
    y_pred = []
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.numpy()
        outputs = model(images)
        _, pred = torch.max(outputs.data, 1)
        pred = pred.detach().cpu().numpy()

        y_true = np.append(y_true, labels)
        y_pred = np.append(y_pred, pred)

    return y_true, y_pred


# check confusion matrix for error analysis
y_true, y_pred = Ture_and_Pred(val_loader, cnn_model)

print(classification_report(y_true, y_pred), '\n\n')
cm = confusion_matrix(y_true, y_pred)


In [ ]:
# Confusion Matrix Plotting Function
def show_confusion_matrix(cm, CLA_label, title='Confusion matrix', cmap=plt.cm.YlGnBu):

    plt.figure(figsize=(10,7))
    plt.grid(False)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(CLA_label))

    plt.xticks(tick_marks, [f"{value}={key}" for key , value in CLA_label.items()], rotation=45)
    plt.yticks(tick_marks, [f"{value}={key}" for key , value in CLA_label.items()])

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, f"{cm[i,j]}\n{cm[i,j]/np.sum(cm)*100:.2f}%", horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.show()

show_confusion_matrix(cm, CLA_label)



### Save Model

In [ ]:
torch.save(cnn_model, "Brain_Tumor_model.pt")
